In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
import theano.tensor as tt

In [ ]:
import scipy.stats as st

As = np.array((1, 3, 10, 12), dtype=float)

def get_pa(p):
    if not np.allclose(p.sum(), 1):
        print(p, p.sum())
        raise ValueError()
    return sum(As * p)

st.binom(10, 0.1)

In [ ]:
p_ = np.random.randn(4)
norm = lambda p: p / p.sum()
p_ /= sum(p_)
data = [{'pa': get_pa(norm(i)), **{f'var_{j}': val for j, val in enumerate(norm(i))}} for i in np.random.random(size=(1000, 4))]
data = pd.DataFrame(data)
data

In [ ]:
def gen_counts(N, ps, bar=None):
    s = 0
    if bar:
        bar.update(1)
    while s != N: 
        d = pd.Series(np.concatenate([st.binom(N, p).rvs(1) for p in ps]), ps.index)
        s = d.sum()
    return d

from tqdm import tqdm
tqdm.pandas('Creating counts')
with tqdm(data.index) as bar:
    data2 = data.apply(lambda row: gen_counts(30, row.iloc[1:], bar=bar), axis=1).assign(pa=data['pa'])

In [ ]:
display(data)
data2

In [ ]:
ps = data2[['var_0', 'var_1', 'var_2', 'var_3']] / 30
err = ps * (1 - ps)
err

In [ ]:
with pm.Model() as model:
    ps = data2[['var_0', 'var_1', 'var_2', 'var_3']] / 30
    ps
    ps = pm.Uniform('ps', lower=0, upper=1, shape=4)
    var = 30 * ps
    print(var)
#     A = pm.Cauchy('A', shape=4)
#     pa = pm.Normal('l', tt.sum(A * var, axis=1), observed=data2.pa.values)